In [3]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment, Tag

In [4]:

import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString#, Comment
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')

        return date
    else:
        'No date found'    
#convert time string to datetime object
def convert_time_string(posted_date):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern = r'\b\d{2}/\d{2}/\d{4}\b'

    # Search for the pattern in the string and extract the date
    match = re.search(pattern, posted_date)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        return date_obj
    else:
        return None 
def get_time_string(date_str):
    if len(date_str)<=5:
        crawl_time = datetime.fromtimestamp(time.time())
        year = str(crawl_time.year)
        date_str = date_str+"/"+year
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date =  date_obj.strftime('%Y-%m-%d')
    else:
        date = convert_string(date_str)
        date_obj = datetime.strptime(date,'%Y-%m-%d')
    return date_obj, date


def get_content_autodaily(url):
    # def get_content_autodaily()
    response = requests.get(url)
    time.sleep(5)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1').text.strip()
    date = soup.find('time').text.strip()
    published_date = convert_string(date)
    article = soup.find('div', class_ = 'article-detail')
    for script_or_style in article(['script', 'style','iframe']):
                script_or_style.decompose()
    related_item = article.find('div', class_ = 'item-relate')
    related_item.decompose()
    author_item = article.find_all(recursive = True)[-2:]
    for i in author_item:
        i.decompose()
    caption_text_list = article.find_all('em')
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    #for i in article.find_all('img'):
        #i.attrs = ['class', 'alt', 'src', 'data-original']
        #del i['onclick']
        #del i['style']
        #del i['class']
        #del i['alt']
    #remove all image attributes except somes from list
    list_attr = ['src','alt','data-src']
    for i in article.find_all('img'):
        for j in list(i.attrs.keys()):
            if j not in list_attr:
                i.attrs.pop(j)
    img_list = article.find_all('img')
    n_img = len(img_list)
    #print(len(caption_text_list))
    for i in range(0,n_img):

        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        try:
            caption_text = NavigableString(caption_text_list[i].get_text())
            caption_end = NavigableString("[/caption]")
            # Insert the custom tags and caption text around the <img> tag
            img_list[i].insert_before(caption_start)
            img_list[i].insert_after(caption_end)
            img_list[i].insert_after(caption_text) 
        except IndexError as e:
            print(e)
        
    for i in article.find_all('img'):
        i['src'] = i['data-src']
    for i in caption_text_list:
        i.decompose()
        #print(img_list[i]['src'])
        #caption_text_list[i].decompose()
    for i in article.find_all(recursive = True):
        try:
            del i['onclick']
            i['id'] =''
            i['class']=''
            i['style'] =''
            i['href'] = ''
        except AttributeError:
            continue
        #try:
            #print(i)
            
        except TypeError:
            continue
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        i['width'] = 800
        i['height'] = 400
        del i['data-src']
    for item in article.find_all('div'):
        if item.string =="":
            item.decompose()
    article.find_all('strong', recursive = True)[-1].decompose()
    # Append the <i> tag as the last child of the <article> tag
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: autodaily.vn"  # Set the content of <i> tag
    article.append(source_tag)
    #Highlight the fist paragraph by using <strong> tag
    p_tag_1st = article.find_all('p')[0]
    p_text = p_tag_1st.get_text()
    # Create a new <strong> tag with the same text
    strong_tag = soup.new_tag("strong")
    strong_tag.string = p_text
    # Replace the <p> tag with the <strong> tag
    p_tag_1st.replace_with(strong_tag)
    
    #Handling a tag 
    a_tags = soup.find_all('a')
    for a_tag in a_tags:
        # Check if the <a> tag has no child tags
        if all(not isinstance(child, Tag) for child in a_tag.children):
            # Convert <a> tag to its text if it has no child tags
            a_tag.replace_with(a_tag.get_text())
        else:
            # If <a> tag has child elements, replace it with a <span> tag but keep the children
            new_span = soup.new_tag("span")
            new_span.extend(a_tag.contents)  # Use extend to add all child elements
            a_tag.replace_with(new_span)
    return article, title, published_date
def get_post(url):
    try:
        content,title,published_date = get_content_autodaily(url)
        return content,title,published_date
    except AttributeError as e:
        print(e)
def get_list_url(cate_url):
    response = requests.get(cate_url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')
    featured_posts = soup.find('ul', class_ = 'late-news-lst')
    list = featured_posts.find_all('li', class_ = 'clearfix')
    urls = []
    for i in list:
        url = i.find('a')['href']
        urls.append(url)
    return urls
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-1*24*3600)
    for i in urls:
        response = requests.get(i)
        time.sleep(2)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            #wrapper = soup.find('div', class_ = "col780 left clearafter")
            #date_posted = wrapper.find('p',class_ ='news-time left').text.strip()
            #date_posted_norm = convert_time_string(date_posted)
            date_str = soup.find('time').text.strip()
            #print(date_str)
            date_posted_norm = get_time_string(date_str)[0]
            #if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
            if date_posted_norm >= datetime.combine(crawl_time, datetime.min.time()):
                filtered_urls.append(i)
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
#add list url to json
#add list url to json
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_autodaily():
    _autodaily= {
            "home_page":"https://autodaily.vn/",
            "urls":{
                "Ô tô":
                {
                 "url":"https://autodaily.vn/#",
                 "sub-category":{  
                    0:{"name":"Ô tô",
                     "url":"https://autodaily.vn/",
                     "cate_id":58,
                      "url_list" : []},
                 }
                }
            }
        }
#
    add_list(_autodaily)
    add_post(_autodaily)
    return _autodaily
#send post content to wordpress via endpoint
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": '5goalvodichcmnl',#'draftpost',#  # Replace with your actual access token
        "published_date": published_date,
        "domain":"autodaily"
          # Replace with the actual category ID as required
    }
    
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _autodaily = get_news_autodaily()
    for i in list(_autodaily['urls'].keys()):
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_autodaily['urls'][i]['sub-category']):
            url_list =  _autodaily['urls'][i]['sub-category'][j]['url_list']
            print(url_list)
            for t in range(0,len(url_list)):
                content = _autodaily['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _autodaily['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _autodaily['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _autodaily['urls'][i]['sub-category'][j]['cate_id']
                print(title, url_list[t])
                #send_post_to_5goals(title,str(content), cate_id, published_date)
                try:
                    text_len = len(content.text)
                    if text_len <500:
                        print(content.text)
                        continue
                    else:
                         send_post_to_5goals(title,str(content),cate_id,published_date)
                except (AttributeError,TypeError):
                    continue
#if __name__ == '__main__':
   # main()

In [3]:
article = get_list_url('https://autodaily.vn/')

In [4]:
article

['https://autodaily.vn/2023/12/sap-dien-ra-ngay-hoi-trai-nghiem-hyundai-experience-day-2023',
 'https://autodaily.vn/2023/12/vinfast-dai-dien-dong-nam-a-tham-luan-tai-dien-dan-thuong-mai-ben-vung-cop28',
 'https://autodaily.vn/2023/12/chang-6-arrc-2023-man-trinh-dien-xuat-sac-nhat-cua-honda-racing-vietnam',
 'https://autodaily.vn/2023/12/vinfast-vf-7-plus-tan-binh-gay-sot-xinh-xich-thoi-gian-vua-qua-co-dang-mua',
 'https://autodaily.vn/2023/12/thuc-te-hyundai-tucson-2024-an-tuong-hon-nhung-van-kho-canh-tranh-cx-5',
 'https://autodaily.vn/2023/12/nhieu-uu-dai-hap-dan-khi-mua-xe-mitsubishi-trong-thang-122023',
 'https://autodaily.vn/2023/12/thaco-auto-tang-goi-dich-vu-dac-biet-danh-cho-cac-mau-xe-bmw-dip-cuoi-nam',
 'https://autodaily.vn/2023/12/thuc-dung-nhu-toyota-tacoma-2024-loa-xe-co-the-thao-roi-de-giai-tri-rieng',
 'https://autodaily.vn/2023/12/toyota-he-lo-mau-xe-crossover-do-thi-chay-dien-moi',
 'https://autodaily.vn/2023/12/kia-sonet-nang-cap-lo-dien-truoc-ngay-ra-mat-pha-hoi-no

In [6]:
get_content_autodaily('https://autodaily.vn/2023/12/chop-ngay-co-hoi-mua-xe-dien-vinfast-voi-loat-uu-dai-len-toi-125-trieu-dong')[0]

list index out of range


<div class="article-detail">
<div class="" href="" id="" style="">
<strong>Chiết khấu lên tới 125 triệu đồng cho VF 8; quà tặng trị giá tới 116,4 triệu đồng cho VF 9 phiên bản 6 chỗ và 68,4 triệu đồng cho VF e34; giảm 30 triệu đồng kèm một năm sạc pin miễn phí tại trạm sạc công cộng cho khách hàng đặt cọc VF 7 là loạt chính sách ưu đãi đặc biệt cho các dòng xe điện đang được VinFast triển khai đồng thời trong tháng 12.</strong>

</div>
<p class="" href="" id="" style=""><strong class="" href="" id="" style="">“</strong><strong class="" href="" id="" style="">Hai năm không chi phí</strong><strong class="" href="" id="" style="">”</strong><strong class="" href="" id="" style=""> với</strong><strong class="" href="" id="" style=""> VF 9 và</strong><strong class="" href="" id="" style=""> VF e34</strong></p>
<p class="" href="" id="" style="">Để giúp khách hàng tiếp cận các dòng xe điện thông minh, đẳng cấp trong tháng cuối năm 2023, VinFast vừa công bố triển khai chương trình ưu đãi “Hai 

In [4]:
main()

Ô tô 0 https://autodaily.vn/
Ô tô 0 58 Ô tô Ô tô Kia Sonet nâng cấp lộ diện trước ngày ra mắt, phả hơi nóng lên Toyota Raize https://autodaily.vn/2023/12/kia-sonet-nang-cap-lo-dien-truoc-ngay-ra-mat-pha-hoi-nong-len-toyota-raize
Ô tô 0 58 Ô tô Ô tô Tập đoàn Tài Chính Phát triển Quốc tế Mỹ ký Ý định Thư tài trợ 500 triệu USD cho VinFast https://autodaily.vn/2023/12/tap-doan-tai-chinh-phat-trien-quoc-te-my-ky-y-dinh-thu-tai-tro-500-trieu-usd-cho-vinfast
Ô tô 0 58 Ô tô Ô tô Rolls-Royce Cullinan 2024 bản nâng cấp lộ diện, thay đổi khá lạ lùng https://autodaily.vn/2023/12/rolls-royce-cullinan-2024-ban-nang-cap-lo-dien-thay-doi-kha-la-lung


KeyboardInterrupt: 